In [1]:
import os
import pandas as pd
from tqdm import tqdm 
tqdm.pandas()  # This enables progress tracking for pandas
#from sentence_transformers import SentenceTransformer
import time
from lxml import etree
import pathlib
import re
from bs4 import BeautifulSoup
import time
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
pd.set_option('display.max_colwidth', None)

In [2]:
base_url = "https://nextprocurement.bsc.es/api/place/XXX/"
url_menors = "https://nextprocurement.bsc.es/api/place_menores/XXX/"

In [3]:
path = "/export/data_ml4ds/NextProcurement/PLACE/Septiembre2024"
output_path = "/export/data_ml4ds/NextProcurement/PLACE/Diciembre_2024_procesados_all"

In [4]:
all_extracted_spa = "/export/data_ml4ds/NextProcurement/PLACE/pliegos_objectives/all_extracted_21_oct_es"
all_extracted_trad = "/export/data_ml4ds/NextProcurement/PLACE/pliegos_objectives/all_extracted_31_oct_tr"

In [5]:
print(f"-- -- Reading from BSC starts")
all_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.parq')]
df_all = []
for file in tqdm(all_files, desc="Procesando archivos"):
    df = pd.read_parquet(file)
    df_all.append(df)
df_bsc = pd.concat(df_all)

-- -- Reading from BSC starts


Procesando archivos: 100%|██████████| 17/17 [16:38<00:00, 58.75s/it]


In [6]:
#list(df_bsc.doc_name.unique())

In [7]:
len(df_bsc)

4114855

In [8]:
len(df_bsc.procurement_id.unique())

816343

In [9]:
df = pd.read_csv("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Text_Object/processed_results_with_admin_tech_from_source.csv")
df

/tmp/ipykernel_2053163/1367144497.py:1: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Text_Object/processed_results_with_admin_tech_from_source.csv")


,procurement_id,doc_name,admin,tech,is_minor,admin_from_api,tech_from_api
0,ntp00000009,['ntp00000009_Pliego_Prescripciones_tecnicas_URI'\n 'ntp00000009_Pliego_clausulas_administrativas_URI'],ntp00000009_Pliego_clausulas_administrativas_URI,ntp00000009_Pliego_Prescripciones_tecnicas_URI,False,02_PCAP OBRAS SIMPLIFICADO 049 18.pdf,00 PROXECTO 049 18 OBRAS CEMITERIO.rar
1,ntp00000013,['ntp00000013_Pliego_clausulas_administrativas_URI'\n 'ntp00000013_Pliego_Prescripciones_tecnicas_URI'],ntp00000013_Pliego_clausulas_administrativas_URI,ntp00000013_Pliego_Prescripciones_tecnicas_URI,False,Plecdeclusulesadminist-630397-001003PCA_STD_CA.pdf,Plecdeprescripcionstcn-592934-001004PPT_STD_CA.pdf
2,ntp00000020,['ntp00000020_Pliego_Prescripciones_tecnicas_URI'\n 'ntp00000020_Pliego_clausulas_administrativas_URI'],ntp00000020_Pliego_clausulas_administrativas_URI,ntp00000020_Pliego_Prescripciones_tecnicas_URI,False,PLIEGO ADMINISTRATIVAS colonias 2019.pdf,PLIEGO TECNICAS Colonias 2019.pdf
3,ntp00000021,['ntp00000021_Pliego_clausulas_administrativas_URI'\n 'ntp00000021_Pliego_Prescripciones_tecnicas_URI'],ntp00000021_Pliego_clausulas_administrativas_URI,ntp00000021_Pliego_Prescripciones_tecnicas_URI,False,CCP cursos y talleres.pdf,PPT cursos y talleres.pdf
4,ntp00000023,['ntp00000023_Pliego_clausulas_administrativas_URI'\n 'ntp00000023_Pliego_Prescripciones_tecnicas_URI'],ntp00000023_Pliego_clausulas_administrativas_URI,ntp00000023_Pliego_Prescripciones_tecnicas_URI,False,PTCAP Servicios.pdf,PPT limpieza DTV.pdf
...,...,...,...,...,...,...,...
816338,ntp11804573,['ntp11804573_Pliego_Prescripciones_tecnicas_URI'\n 'ntp11804573_Publicaciones_oficiales_URI'\n 'ntp11804573_Pliego_clausulas_administrativas_URI'],ntp11804573_Pliego_clausulas_administrativas_URI,ntp11804573_Pliego_Prescripciones_tecnicas_URI,True,NaN,NaN
816339,ntp11804574,['ntp11804574_Pliego_Prescripciones_tecnicas_URI'\n 'ntp11804574_Pliego_clausulas_administrativas_URI'\n 'ntp11804574_Publicaciones_oficiales_URI'],ntp11804574_Pliego_clausulas_administrativas_URI,ntp11804574_Pliego_Prescripciones_tecnicas_URI,True,NaN,NaN
816340,ntp11804575,['ntp11804575_Pliego_clausulas_administrativas_URI'\n 'ntp11804575_Pliego_Prescripciones_tecnicas_URI'\n 'ntp11804575_Publicaciones_oficiales_URI'],ntp11804575_Pliego_clausulas_administrativas_URI,ntp11804575_Pliego_Prescripciones_tecnicas_URI,True,NaN,NaN
816341,ntp11804576,['ntp11804576_Pliego_Prescripciones_tecnicas_URI'\n 'ntp11804576_Pliego_clausulas_administrativas_URI'\n 'ntp11804576_Publicaciones_oficiales_URI'],ntp11804576_Pliego_clausulas_administrativas_URI,ntp11804576_Pliego_Prescripciones_tecnicas_URI,True,NaN,NaN


In [10]:
#Pliego_Cl\u00e1usulas_Administrativas_17-243-MA-SE.pdf
df_bsc[df_bsc.doc_name.str.contains("17-243-MA-SE.pdf")]

,procurement_id,doc_name,content,alternative_lang,translated_content


In [19]:
len(df[df["tech"].isna()]) / len(df)

0.42463155805831615

In [17]:
len(df)

816343

In [12]:
df[df["tech_from_api"].isna()]

,procurement_id,doc_name,admin,tech,is_minor,admin_from_api,tech_from_api
11,ntp00000098,['ntp00000098_Pliego_clausulas_administrativas_URI'\n 'ntp00000098_Pliego_Prescripciones_tecnicas_URI'],ntp00000098_Pliego_clausulas_administrativas_URI,ntp00000098_Pliego_Prescripciones_tecnicas_URI,False,NaN,NaN
15,ntp00000137,['ntp00000137_Pliego_Prescripciones_tecnicas_URI'\n 'ntp00000137_Pliego_clausulas_administrativas_URI'],ntp00000137_Pliego_clausulas_administrativas_URI,ntp00000137_Pliego_Prescripciones_tecnicas_URI,False,NaN,NaN
25,ntp00000163,['ntp00000163_Pliego_clausulas_administrativas_URI'\n 'ntp00000163_Pliego_Prescripciones_tecnicas_URI'],ntp00000163_Pliego_clausulas_administrativas_URI,ntp00000163_Pliego_Prescripciones_tecnicas_URI,False,NaN,NaN
36,ntp00000180,['ntp00000180_Pliego_Prescripciones_tecnicas_URI'\n 'ntp00000180_Pliego_clausulas_administrativas_URI'],ntp00000180_Pliego_clausulas_administrativas_URI,ntp00000180_Pliego_Prescripciones_tecnicas_URI,False,NaN,NaN
37,ntp00000181,['ntp00000181_Pliego_Prescripciones_tecnicas_URI'\n 'ntp00000181_Pliego_clausulas_administrativas_URI'],ntp00000181_Pliego_clausulas_administrativas_URI,ntp00000181_Pliego_Prescripciones_tecnicas_URI,False,NaN,NaN
...,...,...,...,...,...,...,...
816338,ntp11804573,['ntp11804573_Pliego_Prescripciones_tecnicas_URI'\n 'ntp11804573_Publicaciones_oficiales_URI'\n 'ntp11804573_Pliego_clausulas_administrativas_URI'],ntp11804573_Pliego_clausulas_administrativas_URI,ntp11804573_Pliego_Prescripciones_tecnicas_URI,True,NaN,NaN
816339,ntp11804574,['ntp11804574_Pliego_Prescripciones_tecnicas_URI'\n 'ntp11804574_Pliego_clausulas_administrativas_URI'\n 'ntp11804574_Publicaciones_oficiales_URI'],ntp11804574_Pliego_clausulas_administrativas_URI,ntp11804574_Pliego_Prescripciones_tecnicas_URI,True,NaN,NaN
816340,ntp11804575,['ntp11804575_Pliego_clausulas_administrativas_URI'\n 'ntp11804575_Pliego_Prescripciones_tecnicas_URI'\n 'ntp11804575_Publicaciones_oficiales_URI'],ntp11804575_Pliego_clausulas_administrativas_URI,ntp11804575_Pliego_Prescripciones_tecnicas_URI,True,NaN,NaN
816341,ntp11804576,['ntp11804576_Pliego_Prescripciones_tecnicas_URI'\n 'ntp11804576_Pliego_clausulas_administrativas_URI'\n 'ntp11804576_Publicaciones_oficiales_URI'],ntp11804576_Pliego_clausulas_administrativas_URI,ntp11804576_Pliego_Prescripciones_tecnicas_URI,True,NaN,NaN
